## Creating Large Dataset for Model Training
This s comprised of momentum indicators of several handpicked stocks that I think represent a good space in the U.S.A market, indecies that the U.S.A market depend on, and commodities that the the U.S.A depends on. 

In [1]:
### Importing Needed Packages
import pandas as pd
import yfinance as yf
import pandas_ta as ta

In [2]:
### Function SPace

# This on will append all needed momentum indicators to the dataframe provided
def momentum_indicators(df):
    df.ta.rsi(append=True)
    df.ta.macd(append=True)
    df.ta.roc(append=True)
    df.ta.willr(append=True)
    df.ta.stochrsi(append=True)
    df.ta.inertia(append=True)
    df.ta.dm(append=True)
    return df

# This on willl add the indicators from the above function and then remove the unwanted pieces
def add_indicators(data):
    for key in data.keys():
        data[key] = momentum_indicators(data[key])
        drops = ["Adj Close", "High", "Open", "Low", "Close", "Volume"]
        data[key].drop(columns=drops, inplace=True)
    return data

# This one will create the target data from the S&P500 index
def make_target(SP500, ema_len=30):
    SP500.ta.ema(length = 30, append=True)
    SP500["EMA_30_FT"] = SP500.EMA_30.shift(periods=-30)
    SP500["Diff"] = (SP500.EMA_30_FT - SP500.EMA_30)
    SP500["Diff_ratio"] = SP500.Diff / SP500.EMA_30
    SP500["Diff_shift"] = SP500.Diff_ratio.shift(periods=-30)
    SP500["Target"] = SP500.Diff_shift.apply(lambda x: -1 if x < -0.01 else (1 if x > 0.01 else 0))
    return SP500.Target

---
### Downloading Data

Packages and Functions are loaded and now to grab the data. The code below will download all the **ticker** symbols using a yahoo fiance scraper called yfinance. Its a great package to have. I highly recommend it. The data will come in a as dataframe with columns for Open, High, Low, Close, Volume, and Adjusted Close. My code will the pandas dataframes into a dictionary for storage.  

In [3]:
data = {}
tickers = {}
tickers["company_tickers"] = ["JNJ", "PFE", "PG", "MSFT", "AMZN", "CVX", "XOM", "BP", "INTC", "AMD", "BA", "T"]
tickers["index_tickers"] = ["^GSPC", "^IXIC", "^DJI",  "000001.SS", "^N225"]
tickers["commodities_tickers"] = ["CL=F", "HG=F", "GC=F", "SI=F", "RB=F", "NG=F"]

print(f"Loading Companies")
for company in tickers["company_tickers"]:
    data[company] = yf.download(company, start="2000-01-01", end="2021-11-30", interval="1D")
    

print(f"Loading Indecies")
for index in tickers["index_tickers"]:
    data[index] = yf.download(index, start="2000-01-01", end="2021-11-30", interval="1D")


print(f"Loading Commodities Futures")
for commodity in tickers["commodities_tickers"]:
    data[commodity] = yf.download(commodity, start="2000-01-01", end="2021-11-30", interval="1D")

Loading Companies
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
Loading Indecies
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

The Data is downloaded now we need the indicators!

---
### Creating the Dataset

Adding the indicators

In [4]:
data = add_indicators(data)

Gluing together all the dataframes created.

In [5]:
dataframe = pd.concat([data[key] for key in data.keys()], axis=1)
dataframe.shape

(5699, 253)

Downloading the Target Data!

In [6]:
SPY = yf.download("^GSPC", start="2000-01-01", end="2021-11-30", interval="1D")

[*********************100%***********************]  1 of 1 completed


Making the Target column

In [7]:
target = make_target(SPY)

In [8]:
target.shape

(5513,)

Appending the Target Column!

In [9]:
dataframe = pd.concat([dataframe, target], axis=1)
dataframe.dropna(axis=0, inplace=True)
dataframe.shape

(4620, 254)

In [10]:
dataframe.head(5)

,RSI_14,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,ROC_10,WILLR_14,STOCHRSIk_14_14_3_3,STOCHRSId_14_14_3_3,INERTIA_20_14,DMP_14,...,MACDh_12_26_9,MACDs_12_26_9,ROC_10,WILLR_14,STOCHRSIk_14_14_3_3,STOCHRSId_14_14_3_3,INERTIA_20_14,DMP_14,DMN_14,Target
Date,,,,,,,,,,,,,,,,,,,,,
2001-01-09,47.930677,0.196231,-0.312678,0.508910,-3.017241,-69.364162,14.640782,9.064332,70.408573,3.648233,...,-0.072581,0.668381,2.307127,-19.047606,44.663730,33.882334,63.504511,3.221142,2.204278,-1.0
2001-01-10,41.186181,0.035816,-0.378475,0.414291,-6.349206,-93.063584,10.334525,11.073755,66.031602,3.387645,...,-0.094856,0.644667,-6.326529,-48.571418,41.022250,40.737158,64.665545,2.991060,2.476830,-1.0
2001-01-11,34.228196,-0.227382,-0.513338,0.285957,-10.254854,-98.181818,6.394825,10.456711,60.583955,3.145670,...,-0.139717,0.609738,-13.247011,-70.952389,26.610879,37.432287,63.552753,2.777413,2.719913,-1.0
2001-01-12,40.877630,-0.346242,-0.505759,0.159517,-9.725537,-82.727273,5.832570,7.520640,57.736051,2.920980,...,-0.181258,0.564423,-8.539344,-82.285722,9.551023,25.728051,60.483361,2.579027,2.905634,-1.0
2001-01-16,40.107042,-0.450377,-0.487915,0.037538,-10.350982,-85.454545,10.989216,7.738870,53.744497,2.774838,...,-0.226488,0.507801,-17.104859,-91.973700,1.656342,12.606081,55.650165,2.394810,2.978088,-1.0


This is the Data! Now I'll save it as a csv file.

In [11]:
dataframe.to_csv(r"data/momentum_market_data.csv")